# Static heat equation on hex mesh
[![Google Collab Book](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/meyer-nils/torch-fem/blob/main/examples/basic/solid/cubes.ipynb)

Solving the static heat eqaution on a hexahedron mesh.

In [1]:
import meshio as mo
import torch

from torchfem import SolidHeat
from torchfem.examples import get_example_file
from torchfem.materials import IsotropicConductivity3D

# Set default data type to double precision
torch.set_default_dtype(torch.float64)

# Elastic material model
material = IsotropicConductivity3D(kappa=1000.0)

## 3D unstructured mesh with linear hexahedrons read from Abaqus input file 


In [2]:
inp_path = get_example_file("extruded_hex_mesh.inp")

mesh = mo.read(inp_path)
# Create model
nodes = torch.tensor(mesh.points)
elements = torch.tensor(mesh.cells_dict["hexahedron"])
box_heat = SolidHeat(nodes, elements, material)

# Assign boundary conditions
box_heat.constraints[mesh.point_sets["temp_left"]] = True
box_heat.constraints[mesh.point_sets["temp_right"]] = True
# box_heat.constraints[mesh.point_sets["insulation"]] = True
box_heat.displacements[mesh.point_sets["temp_left"], 0] = 10.0
box_heat.displacements[mesh.point_sets["temp_right"], 0] = 20.0
# box_heat.forces[nodes[:, 1] == 1.0, 0] = 0.0
box_heat.forces[mesh.point_sets["insulation"]] = 5.0


# Solve
u, f, sigma, F, _ = box_heat.solve()

box_heat.plot(node_property={"Temperature": u}, show_undeformed=True, opacity=1, colormap="magma")


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…